<a href="https://colab.research.google.com/github/sgraziano26/Coursera_Capstone/blob/master/CourseraCapstoneFeatureResearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier

In [48]:
df = pd.read_csv("Collisions.csv")
df.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (35) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,LOCATION,EXCEPTRSNCODE,EXCEPTRSNDESC,SEVERITYCODE,SEVERITYDESC,COLLISIONTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,INJURIES,SERIOUSINJURIES,FATALITIES,INCDATE,INCDTTM,JUNCTIONTYPE,SDOT_COLCODE,SDOT_COLDESC,INATTENTIONIND,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,-122.344896,47.717173,1,1003,1003,3503158,Matched,Block,NaN,AURORA AVE N BETWEEN N 117TH PL AND N 125TH ST,,NaN,2,Injury Collision,Cycles,2,0,1,1,1,0,0,2013/03/14 00:00:00+00,3/14/2013 17:30,Driveway Junction,18.0,"MOTOR VEHICLE STRUCK PEDALCYCLIST, FRONT END A...",NaN,N,Clear,Dry,Daylight,Y,NaN,NaN,45,Vehicle - Pedalcyclist,15057,0,N
1,-122.376467,47.543774,2,56200,56200,1795087,Matched,Block,NaN,35TH AVE SW BETWEEN SW MORGAN ST AND SW HOLLY ST,NaN,NaN,2,Injury Collision,Pedestrian,7,1,0,1,1,0,0,2006/01/15 00:00:00+00,1/15/2006 17:35,Mid-Block (not related to intersection),24.0,MOTOR VEHCILE STRUCK PEDESTRIAN,NaN,0,Overcast,Dry,Dark - Street Lights On,NaN,6015003.0,NaN,0,Vehicle going straight hits pedestrian,0,0,N
2,-122.360735,47.701487,3,327037,328537,E979380,Matched,Intersection,37122.0,3RD AVE NW AND NW 100TH ST,,NaN,1,Property Damage Only Collision,Angles,2,0,0,2,0,0,0,2019/09/09 00:00:00+00,9/9/2019 15:32,At Intersection (intersection related),14.0,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, REAR END",NaN,N,Overcast,Wet,Daylight,NaN,NaN,NaN,10,Entering at angle,0,0,N
3,-122.297415,47.599233,4,327278,328778,E996362,Unmatched,Intersection,30602.0,M L KING JR WAY S AND S JACKSON ST,,NaN,1,Property Damage Only Collision,NaN,2,0,0,0,0,0,0,2019/12/19 00:00:00+00,12/19/2019,At Intersection (intersection related),14.0,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, REAR END",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,0,0,N
4,-122.368001,47.653585,5,1248,1248,3645424,Unmatched,Block,NaN,W EWING ST BETWEEN 6TH AVE W AND W EWING PL,,NaN,0,Unknown,NaN,1,0,0,0,0,0,0,2013/03/27 00:00:00+00,3/27/2013,Mid-Block (not related to intersection),0.0,NOT ENOUGH INFORMATION / NOT APPLICABLE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,0,0,N


In [49]:
df2 = df.copy()

In [50]:
df2.dtypes

X                  float64
Y                  float64
OBJECTID             int64
INCKEY               int64
COLDETKEY            int64
REPORTNO            object
STATUS              object
ADDRTYPE            object
INTKEY             float64
LOCATION            object
EXCEPTRSNCODE       object
EXCEPTRSNDESC       object
SEVERITYCODE        object
SEVERITYDESC        object
COLLISIONTYPE       object
PERSONCOUNT          int64
PEDCOUNT             int64
PEDCYLCOUNT          int64
VEHCOUNT             int64
INJURIES             int64
SERIOUSINJURIES      int64
FATALITIES           int64
INCDATE             object
INCDTTM             object
JUNCTIONTYPE        object
SDOT_COLCODE       float64
SDOT_COLDESC        object
INATTENTIONIND      object
UNDERINFL           object
WEATHER             object
ROADCOND            object
LIGHTCOND           object
PEDROWNOTGRNT       object
SDOTCOLNUM         float64
SPEEDING            object
ST_COLCODE          object
ST_COLDESC          object
S

In [51]:
#Drop columns
df2.drop(['X','Y','OBJECTID','INCKEY','COLDETKEY','REPORTNO','STATUS','INTKEY','LOCATION','EXCEPTRSNCODE','EXCEPTRSNDESC','SEVERITYDESC','INCDATE','INCDTTM','SDOT_COLDESC','SDOTCOLNUM','ST_COLDESC','SEGLANEKEY','CROSSWALKKEY','ST_COLCODE','SDOT_COLCODE'], axis = 1, inplace=True)

In [52]:
#Change all object fields to lower case for consistency
df2['ADDRTYPE'] = df2['ADDRTYPE'].str.lower()
df2['JUNCTIONTYPE'] = df2['JUNCTIONTYPE'].str.lower()
df2['WEATHER'] = df2['WEATHER'].str.lower()
df2['COLLISIONTYPE'] = df2['COLLISIONTYPE'].str.lower()
df2['ROADCOND'] = df2['ROADCOND'].str.lower()
df2['LIGHTCOND'] = df2['LIGHTCOND'].str.lower()
df2['HITPARKEDCAR'] = df2['HITPARKEDCAR'].str.lower()
df2['SPEEDING'] = df2['SPEEDING'].str.lower()
df2['INATTENTIONIND'] = df2['INATTENTIONIND'].str.lower()
df2['UNDERINFL'] = df2['UNDERINFL'].str.lower()
df2['PEDROWNOTGRNT'] = df2['PEDROWNOTGRNT'].str.lower()

In [53]:
#Turning Flags to Numeric
df2.UNDERINFL.replace(to_replace=['n', 'y'], value=[0, 1], inplace=True)
df2.INATTENTIONIND.replace(to_replace=[ np.NaN, 'y'], value=[0, 1], inplace=True)
df2.HITPARKEDCAR.replace(to_replace=['n', 'y'], value=[0, 1], inplace=True)
df2.SPEEDING.replace(to_replace=['y',  np.NaN], value=[1, 0], inplace=True)
df2.PEDROWNOTGRNT.replace(to_replace=[ np.NaN, 'y'], value=[0, 1], inplace=True)
df2.UNDERINFL =df2.UNDERINFL.astype(float)

In [54]:
#Create target variable and category variables
y = ['SEVERITYCODE']
flags = ['INATTENTIONIND','UNDERINFL','SPEEDING']#,'HITPARKEDCAR','PEDROWNOTGRNT']
cat_columns = ['ROADCOND','LIGHTCOND','WEATHER']#'COLLISIONTYPE','ADDRTYPE','JUNCTIONTYPE',]
numeric_columns = ['VEHCOUNT']#,'FATALITIES','PEDCYLCOUNT','SERIOUSINJURIES','PERSONCOUNT','PEDCOUNT','INJURIES']

In [55]:
from sklearn.impute import SimpleImputer
for f in cat_columns:
  print(f, df2[f].unique())

ROADCOND ['dry' 'wet' nan 'unknown' 'ice' 'other' 'snow/slush' 'standing water'
 'oil' 'sand/mud/dirt']
LIGHTCOND ['daylight' 'dark - street lights on' nan 'dark - no street lights' 'dawn'
 'unknown' 'dark - street lights off' 'dusk' 'other'
 'dark - unknown lighting']
WEATHER ['clear' 'overcast' nan 'raining' 'unknown' 'sleet/hail/freezing rain'
 'other' 'snowing' 'fog/smog/smoke' 'blowing sand/dirt' 'severe crosswind'
 'partly cloudy' 'blowing snow']


In [56]:
#Imputing
imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
imp2 =  SimpleImputer(missing_values=np.nan, strategy='mean')
cat_cols = pd.DataFrame(imp.fit_transform(df2[flags+cat_columns]), columns=flags+cat_columns)
num_cols = pd.DataFrame(imp2.fit_transform(df2[numeric_columns]), columns  = numeric_columns)
fdf = pd.concat([cat_cols, num_cols], axis = 1)

In [57]:
#One Hot Encoding
end = OneHotEncoder(sparse=False)
new_cat = end.fit_transform(fdf[cat_columns])
cat_names = end.get_feature_names(input_features = cat_columns)
cat_col_end = pd.DataFrame(new_cat, columns = cat_names)

In [58]:
#Min Max Scaler
scl = MinMaxScaler()
scl.fit(fdf[numeric_columns])
dat = scl.transform(fdf[numeric_columns])
num_col_scl = pd.DataFrame(dat, columns = numeric_columns)
num_col_scl.head()

,VEHCOUNT
0,0.066667
1,0.066667
2,0.133333
3,0.000000
4,0.000000


In [59]:
#Concatenate into a variable all preprocessed columns
fdf2 = pd.concat([fdf[flags],cat_col_end,num_col_scl], axis = 1)

In [60]:
df2.SEVERITYCODE.replace(to_replace=[np.nan], value=['1'], inplace=True)
df2.SEVERITYCODE.isnull().sum()

0

In [61]:
mdl = RandomForestClassifier()
mdl.fit(fdf2, df2.SEVERITYCODE)
mdl.score(fdf2, df2.SEVERITYCODE)

0.7269054368826854

In [62]:
importance = pd.DataFrame(mdl.feature_importances_)

In [63]:
importance['cols'] = fdf2.columns

In [64]:
importance.columns = ['Imp','cols']
importance.sort_values(by = 'Imp', ascending=False)

,Imp,cols
33,0.794022,VEHCOUNT
10,0.027129,ROADCOND_unknown
17,0.023342,LIGHTCOND_daylight
32,0.021133,WEATHER_unknown
20,0.019610,LIGHTCOND_unknown
23,0.018528,WEATHER_clear
0,0.014538,INATTENTIONIND
3,0.014427,ROADCOND_dry
2,0.012948,SPEEDING
14,0.011227,LIGHTCOND_dark - street lights on
